<!-- # Copyright (c) 2024 Graphcore Ltd. All rights reserved. -->

# Making value tables

In this notebook, we generate value tables akin to those at [P3109](https://htmlpreview.github.io/?https://raw.githubusercontent.com/P3109/Public/main/Value%20Tables/html/index.html).

Thes tables comprise one-line summaries of each float value in the form
```text
Code Binary     = Exact binary E =  Float16 equivalent Float16 binary E    = Float Value
0x21 0_0100_001 = +0b1.001*2^-4  = 0_01011_0010000000 +0b1.0010000000*2^-4 = ~0.0703
```

In [1]:
from gfloat import *
from gfloat.formats import *
import numpy as np
from IPython.display import HTML
import airium

## Define some helpers.

### Render with underscores separating s_e_m

E.g `0_1011_110`.  For formats with zero significand bits or zero exponent bits, we use `0_1011110_` or `0__10111110`.

In [2]:
def str_bits_with_underscores(fi, fv):
    # 0_1011110_
    if fi.tSignificandBits == 0:
        return f"{fv.signbit}_{fv.exp:0{fi.expBits}b}_"

    # 0__1011110
    if fi.expBits == 0:
        return f"{fv.signbit}__{fv.significand:0{fi.tSignificandBits}b}"

    # 0_101_1110
    return (
        f"{fv.signbit}_{fv.exp:0{fi.expBits}b}_{fv.significand:0{fi.tSignificandBits}b}"
    )


fi = format_info_p3109(3)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == "0_10000_01"

fi = format_info_p3109(1)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == "0_1000001_"

fi = format_info_p3109(7)
assert str_bits_with_underscores(fi, decode_float(fi, 0x41)) == "0_1_000001"

### Render a binary16 value

Returns two strings, like this:
```
'0_00010_1010000000', '+0b1.1010000000*2^-13'
```

In [3]:
import struct


def b16_str(val) -> tuple[str, str]:
    """
    Represent VAL in binary16.

    If val does not convert exactly to binary16,
    returns "<Not16:{val}>"
    """
    with np.errstate(over="ignore"):
        b16 = np.float16(val)

    if float(b16) != val and np.isfinite(b16):
        # Finite, but not representable in float16
        return f"<Not16:{val}>", ""
    b16_int = struct.unpack("!H", struct.pack("!e", b16))[0]

    # bitstr is of the form 0_00000_1100000000
    s = f"{b16_int:016b}"
    e_str = s[1:6]
    m_str = s[6:]
    bitstr = f"{s[0]}_{e_str}_{m_str}"

    # pow2str is of the form '+0b0.1100000000*2^-15', or '' for nonfinite values
    e = int(e_str, 2) - 15
    m = int(m_str, 2)
    leading_bit = 0 if e == -15 else 1
    signstr = "-" if s[0] == "1" else "+"
    if np.isfinite(b16):
        pow2str = f"{signstr}0b{leading_bit}.{m:010b}*2^{e}"
    else:
        pow2str = ""
    return bitstr, pow2str


assert b16_str(13 * 2**-16) == ("0_00010_1010000000", "+0b1.1010000000*2^-13")

### Print one table row

In [4]:
def str_tablerow(fi, fv: FloatValue, show_b16_info=True, vs_width=14, vs_d=8):
    """
    Create a string of the form
      0x41 0_10000_01 = +0b1.01*2^0   = 1.25
    optionally adding binary16 info
      0x41 0_10000_01 = +0b1.01*2^0   = 0_01111_0100000000 +0b1.0100000000*2^0 = 1.25
    """
    text = []

    # 0x45 0_1000_101
    text.append(f"0x{fv.code:02x} {str_bits_with_underscores(fi, fv)}")

    finite_nonzero = np.isfinite(fv.fval) and fv.fval != 0

    #  = +0b1.101*2^-7 =
    if finite_nonzero:

        def signstr(fv):
            return "-" if fv.signbit else "+"

        b = "0" if fv.fclass == FloatClass.SUBNORMAL else "1"
        binary_pow2 = f"{signstr(fv)}0b{b}.{fv.significand:0{fi.tSignificandBits}b}*2^{fv.expval:<3}"
        text.append(binary_pow2)

    if show_b16_info and finite_nonzero:
        b16_binary_str, b16_bscistr = b16_str(fv.fval)
        text.append(f"{b16_binary_str} {b16_bscistr}")

    # 1.125
    text.append(float_tilde_unless_roundtrip_str(fv.fval, width=vs_width, d=vs_d))

    # Return tuple
    return " = ".join(text)


for fi in (format_info_p3109(3), format_info_p3109(1)):
    print(fi.name)
    for i in (
        0x00,
        0x01,
        0x07,
        0x21,
        0x40,
        0x41,
        0x7E,
        0x7F,
        0x80,
        0x81,
        0xE6,
        0xFE,
        0xFF,
    ):
        print(
            str_tablerow(
                fi, decode_float(fi, i), show_b16_info=True, vs_width=8, vs_d=4
            )
        )

p3109_p3
0x00 0_00000_00 = 0.0
0x01 0_00000_01 = +0b0.01*2^-15 = 0_00000_0010000000 +0b0.0010000000*2^-15 = ~7.629e-06
0x07 0_00001_11 = +0b1.11*2^-15 = 0_00000_1110000000 +0b0.1110000000*2^-15 = ~5.341e-05
0x21 0_01000_01 = +0b1.01*2^-8  = 0_00111_0100000000 +0b1.0100000000*2^-8 = ~0.0049
0x40 0_10000_00 = +0b1.00*2^0   = 0_01111_0000000000 +0b1.0000000000*2^0 = 1.0
0x41 0_10000_01 = +0b1.01*2^0   = 0_01111_0100000000 +0b1.0100000000*2^0 = 1.25
0x7e 0_11111_10 = +0b1.10*2^15  = 0_11110_1000000000 +0b1.1000000000*2^15 = 49152.0
0x7f 0_11111_11 = inf
0x80 1_00000_00 = nan
0x81 1_00000_01 = -0b0.01*2^-15 = 1_00000_0010000000 -0b0.0010000000*2^-15 = ~-7.629e-06
0xe6 1_11001_10 = -0b1.10*2^9   = 1_11000_1000000000 -0b1.1000000000*2^9 = -768.0
0xfe 1_11111_10 = -0b1.10*2^15  = 1_11110_1000000000 -0b1.1000000000*2^15 = -49152.0
0xff 1_11111_11 = -inf
p3109_p1
0x00 0_0000000_ = 0.0
0x01 0_0000001_ = +0b1.0*2^-62 = <Not16:2.168404344971009e-19>  = ~2.168e-19
0x07 0_0000111_ = +0b1.0*2^-56 = <N

## Make HTML table

In [5]:
def mktbl(fi: FormatInfo, cols=4, skip_rows=None, **kw):
    # Make tables
    nvals = 2**fi.bits
    rows = nvals // cols

    style = f"""
  div.cell_output td {{
    margin: 0pt;
    text-align: left;
  }}

  div.cell_output table {{
    margin: 0pt;
    text-align: left;
    font-family: monospace;
    font-size: xx-small;
    font-weight: bold;
    border-collapse: collapse;
  }}

  
  table {{
    margin: 0pt;
    font-family: monospace;
    font-size: xx-small;
    font-weight: bold;
    border-collapse: collapse;
  }}

  tr.blankrow {{
    height: 4ex;
    vertical-align: top;
  }}
  
  td {{
    text-align: left;
    border: solid 2px #ccc;
    width: {98/cols}%;
  }}
  
  .special {{
    color: #874723;
  }}
  
  .subnormal {{
    color: #0121a7;
  }}
  
  .normal {{
  }}
  
  @media (prefers-color-scheme: dark) {{
    .special {{
      color: orange;
    }}

    .subnormal {{
      color: cyan;
    }}
    
    .normal {{
    }}
  }}

  pre {{
    margin: 1pt 1pt 1pt 13pt;
    display: inline;
  }}
"""

    def table_style(fv):
        """
        Select from the table entry styles defined in CSS above.
        """
        if fv.fclass == FloatClass.SUBNORMAL:
            return "subnormal"

        if fv.fclass == FloatClass.NORMAL:
            return "normal"

        if fv.fclass == FloatClass.ZERO and not fv.signbit:
            return "normal"

        # Everyting else is special
        return "special"

    title = f"FP8 Value Table, {fi.name}"
    a = airium.Airium()
    a.style(_t=style)
    a.h3(_t=title)

    with a.table():
        for i in range(0, rows):
            if skip_rows and (skip_rows[0] <= i < skip_rows[1]):
                if i == skip_rows[0]:
                    a.tr(klass="blankrow").td("...")
                continue
            trklass = "blankrow" if i > 0 and i % 16 == 0 else ""
            with a.tr(klass=trklass):
                for n in range(i, nvals, rows):
                    fv = decode_float(fi, n)
                    text = str_tablerow(fi, fv, show_b16_info=False, **kw)
                    a.td(klass=table_style(fv)).pre(_t=text)

    return str(a)


HTML(mktbl(format_info_ocp_e2m1, cols=2, vs_width=8, vs_d=3))

0x00 0_00_0 = 0.0,0x08 1_00_0 = -0.0
0x01 0_00_1 = +0b0.1*2^0 = 0.5,0x09 1_00_1 = -0b0.1*2^0 = -0.5
0x02 0_01_0 = +0b1.0*2^0 = 1.0,0x0a 1_01_0 = -0b1.0*2^0 = -1.0
0x03 0_01_1 = +0b1.1*2^0 = 1.5,0x0b 1_01_1 = -0b1.1*2^0 = -1.5
0x04 0_10_0 = +0b1.0*2^1 = 2.0,0x0c 1_10_0 = -0b1.0*2^1 = -2.0
0x05 0_10_1 = +0b1.1*2^1 = 3.0,0x0d 1_10_1 = -0b1.1*2^1 = -3.0
0x06 0_11_0 = +0b1.0*2^2 = 4.0,0x0e 1_11_0 = -0b1.0*2^2 = -4.0
0x07 0_11_1 = +0b1.1*2^2 = 6.0,0x0f 1_11_1 = -0b1.1*2^2 = -6.0


### OCP E2M3

This 6-bit format has 32 values, with no `NaN` or `Inf`, but does have `-0`.
The positive subnormals are the linear ramp of eighths: [n/8 for n in 1:7].

One might describe the format in text as:

> zero to one by eighths, two to four by quarters, four to eight by halves

where "to" is open-ended, or "to" is not "thru".

In [6]:
HTML(mktbl(format_info_ocp_e2m3, cols=2, vs_width=8, vs_d=3))

0x00 0_00_000 = 0.0,0x20 1_00_000 = -0.0
0x01 0_00_001 = +0b0.001*2^0 = 0.125,0x21 1_00_001 = -0b0.001*2^0 = -0.125
0x02 0_00_010 = +0b0.010*2^0 = 0.25,0x22 1_00_010 = -0b0.010*2^0 = -0.25
0x03 0_00_011 = +0b0.011*2^0 = 0.375,0x23 1_00_011 = -0b0.011*2^0 = -0.375
0x04 0_00_100 = +0b0.100*2^0 = 0.5,0x24 1_00_100 = -0b0.100*2^0 = -0.5
0x05 0_00_101 = +0b0.101*2^0 = 0.625,0x25 1_00_101 = -0b0.101*2^0 = -0.625
0x06 0_00_110 = +0b0.110*2^0 = 0.75,0x26 1_00_110 = -0b0.110*2^0 = -0.75
0x07 0_00_111 = +0b0.111*2^0 = 0.875,0x27 1_00_111 = -0b0.111*2^0 = -0.875
0x08 0_01_000 = +0b1.000*2^0 = 1.0,0x28 1_01_000 = -0b1.000*2^0 = -1.0
0x09 0_01_001 = +0b1.001*2^0 = 1.125,0x29 1_01_001 = -0b1.001*2^0 = -1.125
0x0a 0_01_010 = +0b1.010*2^0 = 1.25,0x2a 1_01_010 = -0b1.010*2^0 = -1.25


### OCP Formats: E5M2, E4M3

In [7]:
HTML(mktbl(format_info_ocp_e5m2, cols=4, skip_rows=(0x10, 0x30), vs_width=8, vs_d=5))

0x00 0_00000_00 = 0.0,0x40 0_10000_00 = +0b1.00*2^1 = 2.0,0x80 1_00000_00 = -0.0,0xc0 1_10000_00 = -0b1.00*2^1 = -2.0
0x01 0_00000_01 = +0b0.01*2^-14 = ~1.5259e-05,0x41 0_10000_01 = +0b1.01*2^1 = 2.5,0x81 1_00000_01 = -0b0.01*2^-14 = ~-1.5259e-05,0xc1 1_10000_01 = -0b1.01*2^1 = -2.5
0x02 0_00000_10 = +0b0.10*2^-14 = ~3.0518e-05,0x42 0_10000_10 = +0b1.10*2^1 = 3.0,0x82 1_00000_10 = -0b0.10*2^-14 = ~-3.0518e-05,0xc2 1_10000_10 = -0b1.10*2^1 = -3.0
0x03 0_00000_11 = +0b0.11*2^-14 = ~4.5776e-05,0x43 0_10000_11 = +0b1.11*2^1 = 3.5,0x83 1_00000_11 = -0b0.11*2^-14 = ~-4.5776e-05,0xc3 1_10000_11 = -0b1.11*2^1 = -3.5
0x04 0_00001_00 = +0b1.00*2^-14 = ~6.1035e-05,0x44 0_10001_00 = +0b1.00*2^2 = 4.0,0x84 1_00001_00 = -0b1.00*2^-14 = ~-6.1035e-05,0xc4 1_10001_00 = -0b1.00*2^2 = -4.0
0x05 0_00001_01 = +0b1.01*2^-14 = ~7.6294e-05,0x45 0_10001_01 = +0b1.01*2^2 = 5.0,0x85 1_00001_01 = -0b1.01*2^-14 = ~-7.6294e-05,0xc5 1_10001_01 = -0b1.01*2^2 = -5.0
0x06 0_00001_10 = +0b1.10*2^-14 = ~9.1553e-05,0x46 0_10001_10 = +0b1.10*2^2 = 6.0,0x86 1_00001_10 = -0b1.10*2^-14 = ~-9.1553e-05,0xc6 1_10001_10 = -0b1.10*2^2 = -6.0
0x07 0_00001_11 = +0b1.11*2^-14 = ~0.00011,0x47 0_10001_11 = +0b1.11*2^2 = 7.0,0x87 1_00001_11 = -0b1.11*2^-14 = ~-0.00011,0xc7 1_10001_11 = -0b1.11*2^2 = -7.0
0x08 0_00010_00 = +0b1.00*2^-13 = ~0.00012,0x48 0_10010_00 = +0b1.00*2^3 = 8.0,0x88 1_00010_00 = -0b1.00*2^-13 = ~-0.00012,0xc8 1_10010_00 = -0b1.00*2^3 = -8.0
0x09 0_00010_01 = +0b1.01*2^-13 = ~0.00015,0x49 0_10010_01 = +0b1.01*2^3 = 10.0,0x89 1_00010_01 = -0b1.01*2^-13 = ~-0.00015,0xc9 1_10010_01 = -0b1.01*2^3 = -10.0
0x0a 0_00010_10 = +0b1.10*2^-13 = ~0.00018,0x4a 0_10010_10 = +0b1.10*2^3 = 12.0,0x8a 1_00010_10 = -0b1.10*2^-13 = ~-0.00018,0xca 1_10010_10 = -0b1.10*2^3 = -12.0


In [8]:
HTML(mktbl(format_info_ocp_e4m3, cols=4, skip_rows=(0x10, 0x30), vs_width=8, vs_d=5))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^1 = 2.0,0x80 1_0000_000 = -0.0,0xc0 1_1000_000 = -0b1.000*2^1 = -2.0
0x01 0_0000_001 = +0b0.001*2^-6 = ~0.00195,0x41 0_1000_001 = +0b1.001*2^1 = 2.25,0x81 1_0000_001 = -0b0.001*2^-6 = ~-0.00195,0xc1 1_1000_001 = -0b1.001*2^1 = -2.25
0x02 0_0000_010 = +0b0.010*2^-6 = ~0.00391,0x42 0_1000_010 = +0b1.010*2^1 = 2.5,0x82 1_0000_010 = -0b0.010*2^-6 = ~-0.00391,0xc2 1_1000_010 = -0b1.010*2^1 = -2.5
0x03 0_0000_011 = +0b0.011*2^-6 = ~0.00586,0x43 0_1000_011 = +0b1.011*2^1 = 2.75,0x83 1_0000_011 = -0b0.011*2^-6 = ~-0.00586,0xc3 1_1000_011 = -0b1.011*2^1 = -2.75
0x04 0_0000_100 = +0b0.100*2^-6 = ~0.00781,0x44 0_1000_100 = +0b1.100*2^1 = 3.0,0x84 1_0000_100 = -0b0.100*2^-6 = ~-0.00781,0xc4 1_1000_100 = -0b1.100*2^1 = -3.0
0x05 0_0000_101 = +0b0.101*2^-6 = ~0.00977,0x45 0_1000_101 = +0b1.101*2^1 = 3.25,0x85 1_0000_101 = -0b0.101*2^-6 = ~-0.00977,0xc5 1_1000_101 = -0b1.101*2^1 = -3.25
0x06 0_0000_110 = +0b0.110*2^-6 = ~0.01172,0x46 0_1000_110 = +0b1.110*2^1 = 3.5,0x86 1_0000_110 = -0b0.110*2^-6 = ~-0.01172,0xc6 1_1000_110 = -0b1.110*2^1 = -3.5
0x07 0_0000_111 = +0b0.111*2^-6 = ~0.01367,0x47 0_1000_111 = +0b1.111*2^1 = 3.75,0x87 1_0000_111 = -0b0.111*2^-6 = ~-0.01367,0xc7 1_1000_111 = -0b1.111*2^1 = -3.75
0x08 0_0001_000 = +0b1.000*2^-6 = 0.015625,0x48 0_1001_000 = +0b1.000*2^2 = 4.0,0x88 1_0001_000 = -0b1.000*2^-6 = ~-0.01562,0xc8 1_1001_000 = -0b1.000*2^2 = -4.0
0x09 0_0001_001 = +0b1.001*2^-6 = ~0.01758,0x49 0_1001_001 = +0b1.001*2^2 = 4.5,0x89 1_0001_001 = -0b1.001*2^-6 = ~-0.01758,0xc9 1_1001_001 = -0b1.001*2^2 = -4.5
0x0a 0_0001_010 = +0b1.010*2^-6 = ~0.01953,0x4a 0_1001_010 = +0b1.010*2^2 = 5.0,0x8a 1_0001_010 = -0b1.010*2^-6 = ~-0.01953,0xca 1_1001_010 = -0b1.010*2^2 = -5.0


### IEEE WG P3109 {P} formats

We choose just one example: `p3109(p=3)`, which has the same number of exponent bits as OCP E5 

In [9]:
HTML(mktbl(format_info_p3109(3), cols=4, skip_rows=(0x10, 0x30), vs_width=8, vs_d=5))

0x00 0_00000_00 = 0.0,0x40 0_10000_00 = +0b1.00*2^0 = 1.0,0x80 1_00000_00 = nan,0xc0 1_10000_00 = -0b1.00*2^0 = -1.0
0x01 0_00000_01 = +0b0.01*2^-15 = ~7.6294e-06,0x41 0_10000_01 = +0b1.01*2^0 = 1.25,0x81 1_00000_01 = -0b0.01*2^-15 = ~-7.6294e-06,0xc1 1_10000_01 = -0b1.01*2^0 = -1.25
0x02 0_00000_10 = +0b0.10*2^-15 = ~1.5259e-05,0x42 0_10000_10 = +0b1.10*2^0 = 1.5,0x82 1_00000_10 = -0b0.10*2^-15 = ~-1.5259e-05,0xc2 1_10000_10 = -0b1.10*2^0 = -1.5
0x03 0_00000_11 = +0b0.11*2^-15 = ~2.2888e-05,0x43 0_10000_11 = +0b1.11*2^0 = 1.75,0x83 1_00000_11 = -0b0.11*2^-15 = ~-2.2888e-05,0xc3 1_10000_11 = -0b1.11*2^0 = -1.75
0x04 0_00001_00 = +0b1.00*2^-15 = ~3.0518e-05,0x44 0_10001_00 = +0b1.00*2^1 = 2.0,0x84 1_00001_00 = -0b1.00*2^-15 = ~-3.0518e-05,0xc4 1_10001_00 = -0b1.00*2^1 = -2.0
0x05 0_00001_01 = +0b1.01*2^-15 = ~3.8147e-05,0x45 0_10001_01 = +0b1.01*2^1 = 2.5,0x85 1_00001_01 = -0b1.01*2^-15 = ~-3.8147e-05,0xc5 1_10001_01 = -0b1.01*2^1 = -2.5
0x06 0_00001_10 = +0b1.10*2^-15 = ~4.5776e-05,0x46 0_10001_10 = +0b1.10*2^1 = 3.0,0x86 1_00001_10 = -0b1.10*2^-15 = ~-4.5776e-05,0xc6 1_10001_10 = -0b1.10*2^1 = -3.0
0x07 0_00001_11 = +0b1.11*2^-15 = ~5.3406e-05,0x47 0_10001_11 = +0b1.11*2^1 = 3.5,0x87 1_00001_11 = -0b1.11*2^-15 = ~-5.3406e-05,0xc7 1_10001_11 = -0b1.11*2^1 = -3.5
0x08 0_00010_00 = +0b1.00*2^-14 = ~6.1035e-05,0x48 0_10010_00 = +0b1.00*2^2 = 4.0,0x88 1_00010_00 = -0b1.00*2^-14 = ~-6.1035e-05,0xc8 1_10010_00 = -0b1.00*2^2 = -4.0
0x09 0_00010_01 = +0b1.01*2^-14 = ~7.6294e-05,0x49 0_10010_01 = +0b1.01*2^2 = 5.0,0x89 1_00010_01 = -0b1.01*2^-14 = ~-7.6294e-05,0xc9 1_10010_01 = -0b1.01*2^2 = -5.0
0x0a 0_00010_10 = +0b1.10*2^-14 = ~9.1553e-05,0x4a 0_10010_10 = +0b1.10*2^2 = 6.0,0x8a 1_00010_10 = -0b1.10*2^-14 = ~-9.1553e-05,0xca 1_10010_10 = -0b1.10*2^2 = -6.0
